# Importações

In [ ]:
import joblib
import pandas as pd
import shap
from google.colab import drive
import os
import numpy as np

# Configuração para os gráficos aparecerem no notebook
shap.initjs()

#Montar o Google Drive


In [ ]:
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/Ps_Ligia_Time_16/'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Carregar Modelo e Dataset

In [ ]:
'''
Se você executou o notebook de EDA e de Treino deste projeto, e portanto, gerou e salvou os
datasets processados e modelos; Descomente este código abaixo e fique a vontade para
utilizar os datasets que foram salvos no seu próprio Google Drive.

Caso não tenha criado os datasets, utilize o trecho descomentado para carregar
ele já pronto do nosso Drive.
'''

# ----------------------------------------------------------

# model_path = 'models/XGBoost_Features.pkl'

# # Carregar o pipeline completo (que já inclui o preprocessor)
# pipeline = joblib.load(os.path.join(output_path, model_path))

# # Carregar dataset features
# df = pd.read_csv(os.path.join(output_path, 'risco_cardiovascular_features.csv'))

# ----------------------------------------------------------

# Carregando arquivo do modelo via url
import requests
from io import BytesIO
response = requests.get("https://drive.google.com/uc?id=19RcKMhfoqc91zqSyjFyj9uLCdfGpW4Do")
response.raise_for_status()

# Carregar o pipeline completo (que já inclui o preprocessor)
pipeline = joblib.load(BytesIO(response.content))

# Carregar dataset features
df = pd.read_csv("https://drive.google.com/uc?id=142P-9xy-kRboRVpHj2t9OniS0DUF0QTp")

# Separar X e y (para sabermos se ele acertou no final)
X = df.drop('BP_Category', axis=1)
y = df['BP_Category']

# Seleçionar Amostra e Inferência

In [ ]:
labels_map = {
    0: "Pressão Elevada",
    "Elevated": "Pressão Elevada",
    1: "Normal",
    "Normal": "Normal",
    2: "Hipertensão Estágio 1",
    "Stage1": "Hipertensão Estágio 1"
}

# Índice Aleatório
sample_idx = np.random.randint(0, len(X))
sample_X = X.iloc[[sample_idx]]
true_label = y.iloc[sample_idx]

# Predição de Classe e Probabilidade
pred_idx = pipeline.predict(sample_X)[0]
pred_proba = pipeline.predict_proba(sample_X)[0]

print(f"Instância selecionada (índice): {sample_idx}")
print(f"--- Resultado da Inferência ---")
print(f"Classe Real: {labels_map[true_label]}")
print(f"Classe Predita: {labels_map[pred_idx]}")
print(f"Status: {'✅ Acertou' if labels_map[pred_idx] == labels_map[true_label] else '❌ Errou'}")

indices_ordenados = [1, 0, 2]
# Mostrar probabilidades para cada categoria
proba_df = pd.DataFrame({
    'Classe': [labels_map[i] for i in indices_ordenados],
    'Probabilidade Estimada': [pred_proba[i] for i in indices_ordenados]
})
print("\nProbabilidades por classe:")
print(proba_df.to_string(index=False))

# Explicabilidade

In [ ]:
# Dicionário de tradução para as colunas do dataset
tradução_features = {
    "Gender": "Gênero",
    "Age": "Idade",
    "Occupation": "Profissão",
    "Sleep Duration": "Duração do Sono",
    "Quality of Sleep": "Qualidade do Sono",
    "Physical Activity Level": "Nível de Atividade Física",
    "Stress Level": "Nível de Estresse",
    "BMI Category": "Categoria de IMC",
    "Heart Rate": "Freq. Cardíaca",
    "Daily Steps": "Passos Diários",
    "Sleep Disorder": "Distúrbio do Sono",
    "Sleep_Efficiency": "Eficiência do Sono",
    "Cardiac_Stress_Index": "Índice de Estresse Cardíaco"
}

In [ ]:
# Tabela de Valores Paciente
print(f"--- Ficha Técnica do Paciente ---")
perfil_paciente = sample_X.copy()
# Traduz os nomes das colunas para a tabela
perfil_paciente.columns = [tradução_features.get(c, c) for c in perfil_paciente.columns]
# Transpõe para exibição vertical
print(perfil_paciente.T.rename(columns={perfil_paciente.index[0]: 'Valor'}))

# Preparação dos dados para SHAP
preprocessor = pipeline.named_steps['preprocessor']
classifier = pipeline.named_steps['classifier']
X_transformed = preprocessor.transform(sample_X)

# Tradução simples dos nomes para o gráfico
raw_names = preprocessor.get_feature_names_out()
feature_names_pt = []

for name in raw_names:
    clean_name = name.split('__')[-1]
    translated = clean_name
    for eng, pt in tradução_features.items():
        if eng in clean_name:
            translated = clean_name.replace(eng, pt)
            break
    # Melhora visual: remove underscores e ajusta termos técnicos
    translated = translated.replace('_', ' ').replace('Gender Male', 'Gênero: Masc').replace('Gender Female', 'Gênero: Fem')
    feature_names_pt.append(translated)

# SHAP
explainer = shap.TreeExplainer(classifier)
shap_values = explainer.shap_values(X_transformed)

print(f"\nInterpretando os pesos para a decisão: {labels_map[pred_idx]}")

# Gráfico de Barras
shap.plots.bar(shap.Explanation(
    values=shap_values[0, :, pred_idx],
    feature_names=feature_names_pt
))

# Resultados

In [ ]:
# Opcional: Puxar do próprio Drive caso tenha treinado no nosso notebook de treinamento completo
# metrics_file = os.path.join(output_path, 'metrics/XGBoost_Features_metrics.csv')

# Carregando pronto online
metrics_file = "https://drive.google.com/uc?id=1odLsMmUWqimOd7y494XNaRfNRCiFYHYy"

# Carregar os dados
metrics_df = pd.read_csv(metrics_file)

# Dicionário de Tradução "técnico para clínico"
traducao_metricas = {
    'Test_Accuracy': 'Acurácia Geral (Acerto Total)',
    'Test_Precision': 'Confiabilidade do Alerta (Precisão)',
    'Test_Recall': 'Capacidade de Detecção (Sensibilidade)',
    'Test_F1_Macro': 'Equilíbrio Clínico (F1-Score)',
    'Test_AUC_ROC': 'Poder de Discriminação (AUC)',
    'Test_Brier_Score': 'Erro de Calibração (Brier)'
}

# Selecionar e formatar as métricas de interesse
resumo_tecnico = metrics_df[list(traducao_metricas.keys())].iloc[0]

# DataFrame de exibição
tabela_hospitalar = pd.DataFrame({
    'Indicador de Qualidade': traducao_metricas.values(),
    'Desempenho': resumo_tecnico.values
})

# Formatação: Porcentagem para quase tudo, exceto Brier Score
def formatar_valor(row):
    if 'Brier' in row['Indicador de Qualidade']:
        return f"{row['Desempenho']:.4f} (Ideal: próximo a 0)"
    return f"{row['Desempenho']:.2%}"

tabela_hospitalar['Resultado'] = tabela_hospitalar.apply(formatar_valor, axis=1)

print("--- Relatório de Qualidade do Modelo (Contexto Hospitalar) ---")
print(f"Modelo Analisado: {metrics_df['Model'].iloc[0]}")
print("-" * 60)
print(tabela_hospitalar[['Indicador de Qualidade', 'Resultado']].to_string(index=False))

print("\n--- O que esses números significam para o médico? ---")
print(f"• Sensibilidade: O modelo consegue identificar {resumo_tecnico['Test_Recall']:.2%} dos pacientes de risco.")
print(f"• Precisão: Quando o alerta soa, a chance de o risco ser real é de {resumo_tecnico['Test_Precision']:.2%}.")
print(f"• Calibração: O modelo é estatisticamente seguro para apoio à decisão clínica.")